# 开始使用

本节介绍 reki 库的主要功能。

首先导入需要使用的一些库。

In [ ]:
import datetime

import numpy as np
import pandas as pd
import xarray as xr
import eccodes

## 本地文件查找

reki 内置了 CMA 国家级超算子系统 1 共享文件系统中业务系统产品文件路径。

下面示例获取 CMA-GFS 系统 2024 年 7 月 3 日 00 时次 024 时效的原始分辨率 GRIB2 文件路径。

In [ ]:
from reki.data_finder import find_local_file

gfs_grib2_file_path = find_local_file(
    "cma_gfs_gmf/grib2/orig",
    start_time=pd.to_datetime("2024-07-03 00:00"),
    forecast_time=pd.to_timedelta("24h")
)
gfs_grib2_file_path

`find_local_file` 函数支持一些参数来定义更精确的文件查找条件。
比如 `number` 参数表示集合预报成员编号。

下面示例获取 CMA-GEPS 系统 2024 年 7 月 2 日 12 时次第 2 个集合成员 024 时效的原始分辨率 GRIB2 文件路径。

In [ ]:
geps_grib2_file_path = find_local_file(
    "cma_geps/grib2/orig",
    start_time="2024070212",
    forecast_time="24h",
    number=2,
)
geps_grib2_file_path

reki 也内置了模式输出的一些中间文件。

下面示例获取 CMA-GFS 系统 2024 年 7 月 3 日 00 时次 240 时效的模式面二进制数据文件路径。

In [ ]:
gfs_modelvar_file_path = find_local_file(
    "cma_gfs_gmf/bin/modelvar",
    start_time=datetime.datetime(2024, 7, 3, 0, 0),
    forecast_time="24h",
)
gfs_modelvar_file_path

reki 内置的配置文件请查看 `reki/data_finder/conf` 目录。

## 加载 GRIB2 要素场

### 基本方法

reki 使用 eccodes 从 GRIB2 文件中检索要素场，并返回 `xarray.DataArray` 对象。

下面示例从 CMA-GFS 的 GRIB2 产品文件中加载 850hPa 温度场。其中：

- `parameter` 参数表示要素名称，`t` 代表温度
- `level_type` 参数表示层次类型，`pl` 代表等压面层，单位 hPa
- `level` 参数表示层次值

In [ ]:
from reki.format.grib.eccodes import load_field_from_file

field = load_field_from_file(
    gfs_grib2_file_path,
    parameter="t",
    level_type="pl",
    level=850,
)
field

可以使用 xarray 提供的一系列功能对要素场进行分析。比如使用 `xarray.DataArray.plot()` 函数实现快速绘图。

In [ ]:
(field - 273.15).plot()

### 层次类型

reki 支持从模式面数据中加载要素场。

下面代码从 CMA-GFS 的模式面 GRIB2 数据中加载第 10 层 u 分量，其中 `ml` 表示模式面。

In [ ]:
gfs_model_grib2_file_path = find_local_file(
    "cma_gfs_gmf/grib2/modelvar",
    start_time=pd.to_datetime("2024-07-03 00:00"),
    forecast_time=pd.to_timedelta("24h"),
)
gfs_model_grib2_file_path

In [ ]:
field = load_field_from_file(
    gfs_model_grib2_file_path,
    parameter="u",
    level_type="ml",
    level=10,
)
field

### GRIB Key

`load_field_from_file` 函数支持使用 ecCodes 的 GRIB Key 作为参数值。

下面代码通过字典形式的 `parameter` 参数加载 850hPa 雷达反射率场。

In [ ]:
field = load_field_from_file(
    gfs_grib2_file_path,
    parameter={
        "discipline": 0,
        "parameterCategory": 16,
        "parameterNumber": 225,
    },
    level_type="pl",
    level=850
)
field

### ecCodes 接口

reki 支持返回 ecCode Python API 的 GRIB message。

下面代码加载 850hPa 位势高度场。首先获取 GRIB message。

In [ ]:
from reki.format.grib.eccodes import load_message_from_file

message = load_message_from_file(
    gfs_grib2_file_path,
    parameter="gh",
    level_type="pl",
    level=850
)
message

读取经纬度格点数和数据值，将数据重组成二维数组。

In [ ]:
values = eccodes.codes_get_double_array(message, "values")
ni = eccodes.codes_get_long(message, "Ni")
nj = eccodes.codes_get_long(message, "Nj")
values.reshape(nj, ni)